In [15]:
!pip install fake_useragent

In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

In [17]:
input_file= 'F:/Blackcoffer assignment/Input.xlsx'
df=pd.read_excel(input_file)

In [18]:
df.head()

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...


In [19]:
# Function to find if there is any error in URL 
urls = df['URL'].tolist()
status = []

## initializing the UserAgent object
user_agent = UserAgent()

for url in urls:
    try:
        r = requests.get(url,headers={"user-agent": user_agent.chrome})
        r.raise_for_status()
    except (requests.exceptions.ConnectionError, requests.exceptions.Timeout):
        status.append("Down")
    except requests.exceptions.HTTPError:
        status.append("HTTP error")
    else:
        status.append("All good")
df['status'] = status
     

In [24]:
df.loc[df['status']=='HTTP error']

,URL_ID,URL,status
24,11668.0,https://insights.blackcoffer.com/how-neural-ne...,HTTP error
37,17671.4,https://insights.blackcoffer.com/covid-19-envi...,HTTP error


In [23]:
df.loc[df['status']=='All good']

,URL_ID,URL,status
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,All good
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,All good
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,All good
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,All good
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,All good
...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,All good
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,All good
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,All good
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,All good


In [26]:
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    
    retry_count = 0
    max_retries = 3
    
    while retry_count < max_retries:
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raises an exception if HTTP request returned an unsuccessful status code
            soup = BeautifulSoup(response.text, 'html.parser')
            
            article_title = soup.find('title').get_text()  # Extract title
            article_text = ''  # Initialize article text
            
            paragraphs = soup.find_all('p')
            for paragraph in paragraphs:
                article_text += paragraph.get_text() + '\n'
            
            # Save article text to a text file
            with open(f'{url_id}.txt', 'w', encoding='utf-8') as f:
                f.write(article_title + '\n' + article_text)
            
            break  # Break the loop if successful
        except requests.exceptions.ConnectionError as e:
            print(f"Connection error for URL: {url}. Retrying...")
            retry_count += 1
            time.sleep(1)  # Add a small delay before retrying
        except requests.exceptions.RequestException as e:
            print(f"Error for URL: {url}. Skipping...")
            break  # Break the loop if a different error occurs


Error for URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/. Skipping...
Error for URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/. Skipping...
